In [1]:
# Selection: avalia {pop} indivíduos aleatórias e retorna as duas melhores 

# Crossover: cruza dois indivíduos e gera dois novos
#    Particionamento
#    Merge
#    Retirar repetição
   
# Mutation: retorna um subset (2 indivíduos) como resultado da mutação dos filhos 

# Replacement: troca os dois piores 

In [110]:
import random as rd
import numpy as np
import random
import copy
import matplotlib.pyplot as plt

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/jesimon/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2963, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-110-5ba284e0ae43>", line 5, in <module>
    import matplotlib.pyplot as plt
  File "/home/jesimon/anaconda3/lib/python3.6/site-packages/matplotlib/__init__.py", line 1163, in <module>
    rcParams = rc_params()
  File "/home/jesimon/anaconda3/lib/python3.6/site-packages/matplotlib/__init__.py", line 997, in rc_params
    fname = matplotlib_fname()
  File "/home/jesimon/anaconda3/lib/python3.6/site-packages/matplotlib/__init__.py", line 841, in matplotlib_fname
    for fname in gen_candidates():
  File "/home/jesimon/anaconda3/lib/python3.6/site-packages/matplotlib/__init__.py", line 830, in gen_candidates
    yield os.path.join(six.moves.getcwd(), 'matplotlibrc')
FileNotFoundError: [Errno 2] No such file or directory

During handling of the above exception, an

FileNotFoundError: [Errno 2] No such file or directory

# Problema
Resolver o problema de posicionar n rainhas no tabuleiro de Xadrez, sem que as rainhas se coloquem em cheque.

# Funções Implementadas

### init_population 
Gera uma população inicial utilizando a representação genotípica. O tabuleiro (matriz nxn) é representado de tal maneira que cada posição do vetor representa uma linha e a coluna é representada pelo valor no vetor.

### fitness_nq
Calcula o número de cheques que acontecem de acordo com as posições das rainhas.

In [98]:
def init_population(_mu:int = 20, n:int = 8):
    population = []
    for i in range (_mu):
        population.append(rd.sample(range(n), n))
    return population

def fitness_nq(solution):
    xeques = 0
    for i in range(0,len(solution)):
        for j in range(0,len(solution)):
            if i!=j:
                if i-solution[i] == j-solution[j] or i+solution[i] == j+solution[j]:
                    xeques+=1
    return xeques

### Selection
Seleciona um subconjunto das duas melhores soluções presentes na população.

In [99]:
# hipótese sobre erro:
#  Selecionar sempre os dois melhores e cruzá-los não está fazendo com que os genes mais adaptados se espalhem?

def selection(population, cut):
    scores = []
    # escolhendo 5 indivíduos aleatórios     
    for individual in random.choices(population, k=5):
        fitness_score = fitness_nq(individual)
        scores.append((fitness_score, individual))
    sorted_scores = sorted(scores, key=lambda tup: tup[0])
    best_ones = sorted_scores[:cut] 
    return list(map(lambda x: x[1], best_ones))


### Crossover
Faz a junção das duas melhores soluções de maneira aleatoria. Isso é feito selecionando uma posição aleatoria, separa as duas soluções nessa posição e uni o inicio de uma solução ao fim de outra.

In [100]:
def conserta_repeticoes(individual):
    pass

def crossover(subset_parents, cross_over_rate):
    if float(rd.randint(0, 10))/10 <= cross_over_rate:
        size_n = len(subset_parents[0])
        cut_v = random.randint(1, size_n-1)
        son1 = subset_parents[0][0:cut_v] + subset_parents[1][cut_v:]
        son2 = subset_parents[1][0:cut_v] + subset_parents[0][cut_v:]
        control = [] 
        new_son1 = []
        ind = 0
        for n, value in enumerate(son1):
            if value in control:
                new_value = -1
                for i in range(size_n):
                    if i not in new_son1:
                        new_value = i
                        break
                control.append(new_value)
                new_son1.append(new_value)
            else:
                control.append(value)
                new_son1.append(value)

        control = [] 
        new_son2 = []
        ind = 0
        for n, value in enumerate(son2):
            if value in control:
                new_value = -1
                for i in range(size_n):
                    if i not in new_son2:
                        new_value = i
                        break
                control.append(new_value)
                new_son2.append(new_value)
            else:
                control.append(value)
                new_son2.append(value)

    return [new_son1, new_son2]

### Mutation
Muta a população respeitando a taxa de mutação

### Replacement
Substitui os dois piores resultados pelas duas novas soluções geradas

In [101]:
def mutation(population, mutation_rate):
    population_copy = copy.copy(population)
    for individual in population_copy:
        if float(rd.randint(0, 10))*10 <= mutation_rate:
            print("FOI MUTADO!")
            last_gene = individual.pop(0)
            individual.insert(len(individual),last_gene)
    return population_copy


def replacement(offspring_new, pop):
    for i in range(2):
        m = -1
        score_m = 0
        for n,p in enumerate(pop):
            score = fitness_nq(p)
            # mudei de '<' para '>' de forma que fossem tirados os com maior fitness (no. de xeques)          
            if score > score_m:
                m = n
                score_m = score
        del pop[m]
        
    for off in offspring_new:
        pop.append(off)
    
    return pop


# Inicializa Variáveis

In [102]:
pop = init_population(20, 8)
cross_over_rate = 1.0
mutation_rate = 80
sol = []

# Execução do programa

In [104]:
n_iter = 0

fitness_avg = []
best_of_generation = []
smallest_fitness = 1

while n_iter < 100 and smallest_fitness > 0:
    subset_parents = selection(pop, 2)
    offspring = crossover(subset_parents, cross_over_rate)
    offspring_new = mutation(offspring, mutation_rate)
    pop = replacement(offspring_new, pop)
    
    fitness_scores = []
    smallest_fitness = fitness_nq(pop[0])
    
    for p in pop:
        v = fitness_nq(p)
        fitness_scores.append(v)
        if v < smallest_fitness:
            smallest_fitness = v
            sol = copy.copy(p)
            
    best_of_generation.append(smallest_fitness)
            
    current_fitness_avg = sum(fitness_scores)/len(fitness_scores)
    fitness_avg.append(current_fitness_avg)
    print (f"Fitness média da geração {n_iter}: {current_fitness_avg}\n")    
    n_iter += 1

print(f"Solução: {sol}")
print(f"Valor da Solução: {smallest_fitness}")

FOI MUTADO!
FOI MUTADO!
Fitness média da geração 0: 9.1

FOI MUTADO!
FOI MUTADO!
Fitness média da geração 1: 8.2

FOI MUTADO!
FOI MUTADO!
Fitness média da geração 2: 8.0

FOI MUTADO!
FOI MUTADO!
Fitness média da geração 3: 7.5

FOI MUTADO!
FOI MUTADO!
Fitness média da geração 4: 7.4

FOI MUTADO!
FOI MUTADO!
Fitness média da geração 5: 6.7

FOI MUTADO!
FOI MUTADO!
Fitness média da geração 6: 6.8

FOI MUTADO!
FOI MUTADO!
Fitness média da geração 7: 6.2

FOI MUTADO!
FOI MUTADO!
Fitness média da geração 8: 6.3

FOI MUTADO!
FOI MUTADO!
Fitness média da geração 9: 5.7

FOI MUTADO!
FOI MUTADO!
Fitness média da geração 10: 5.6

FOI MUTADO!
FOI MUTADO!
Fitness média da geração 11: 6.1

FOI MUTADO!
Fitness média da geração 12: 5.4

FOI MUTADO!
Fitness média da geração 13: 5.4

FOI MUTADO!
Fitness média da geração 14: 5.5

FOI MUTADO!
FOI MUTADO!
Fitness média da geração 15: 5.3

FOI MUTADO!
Fitness média da geração 16: 5.0

FOI MUTADO!
FOI MUTADO!
Fitness média da geração 17: 5.4

FOI MUTADO!
FO

In [105]:
len(best_of_generation)

23

In [106]:
# Plotting both the curves simultaneously
X = range(n_iter)
plt.plot(X, fitness_avg, color='r', label='Fitness Average')
plt.plot(X, best_of_generation, color='g', label='Best of each gen.')
  
# Naming the x-axis, y-axis and the whole graph
plt.xlabel("Generation")
plt.ylabel("Fitness")
plt.title("Evolution")
  
# Adding legend, which helps us recognize the curve according to it's color
plt.legend()
  
# To load the display window
plt.show()

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/jesimon/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2963, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-106-481378e99a72>", line 3, in <module>
    plt.plot(X, fitness_avg, color='r', label='Fitness Average')
NameError: name 'plt' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/jesimon/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 1863, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/jesimon/anaconda3/lib/python3.6/site-packages/IPython/core/ultratb.py", line 1095, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offs

NameError: name 'plt' is not defined